# Linear VMM (Vessel Maneuvering Model)

# Purpose
Implementing according to:
Matusiak, Jerzy. Dynamics of a Rigid Ship. Aalto University, 2017. https://aaltodoc.aalto.fi:443/handle/123456789/24408.

# Methodology
Define the problem using SymPy

# Setup

In [1]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from src.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
#from src.data import mdl
from src import symbols
from src import prime_system
from src.symbols import *
from src.linear_vmm_equations import *

import src.linear_vmm_equations as eq
import src.models.linear_vmm as model
from src.visualization.plot import track_plot
import src.nonlinear_vmm_equations as nonlinear_vmm_equations


Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 463 ('figure.dpi       : 100        ## figure dots per inch')


In [2]:
symbols.df_parameters

,symbol,dof,coord,state,brix,brix_lambda
Xudot,X_{\dot{u}},X,u,dot,2.0*m/(L**3*rho*(pi*sqrt(L**3/volume) - 14)),<function _lambdifygenerated at 0x000001E4B869...
Xvdot,X_{\dot{v}},X,v,dot,NaN,NaN
Xrdot,X_{\dot{r}},X,r,dot,NaN,NaN
Yudot,Y_{\dot{u}},Y,u,dot,NaN,NaN
Yvdot,Y_{\dot{v}},Y,v,dot,-pi*T**2*(-5.1*B**2/L**2 + 0.16*B*CB/T + 1)/L**2,<function _lambdifygenerated at 0x000001E4B869...
Yrdot,Y_{\dot{r}},Y,r,dot,-pi*T**2*(-0.0033*B**2/T**2 + 0.67*B/L)/L**2,<function _lambdifygenerated at 0x000001E4B868...
Nudot,N_{\dot{u}},N,u,dot,NaN,NaN
Nvdot,N_{\dot{v}},N,v,dot,-pi*T**2*(-0.04*B/T + 1.1*B/L)/L**2,<function _lambdifygenerated at 0x000001E4B869...
Nrdot,N_{\dot{r}},N,r,dot,-pi*T**2*(0.017*B*CB/T - 0.33*B/L + 0.08333333...,<function _lambdifygenerated at 0x000001E4B869...
Xu,X_{u},X,u,,NaN,NaN


## Linearized equation of motion

### X

Nonlinear equation of motion in X-direction:

In [3]:
Math(vlatex(nonlinear_vmm_equations.X_eom))

<IPython.core.display.Math object>

this equation is linearized by assuming : $v=0$ and dropping $r^2$

In [4]:
Math(vlatex(eq.X_eom))

<IPython.core.display.Math object>

### Y

Nonlinear equation of motion in Y-direction:

In [5]:
Math(vlatex(nonlinear_vmm_equations.Y_eom))

<IPython.core.display.Math object>

this equation is linearized by assuming that $u \approx U $ where $U$ is the initial total velocity.

In [6]:
Math(vlatex(eq.Y_eom))

<IPython.core.display.Math object>

### N

Nonlinear equation of motion in N-direction:

In [7]:
Math(vlatex(nonlinear_vmm_equations.N_eom))

<IPython.core.display.Math object>

this equation is linearized by assuming that $u \approx U $ where $U$ is the initial total velocity.

In [8]:
Math(vlatex(eq.N_eom))

<IPython.core.display.Math object>

In [9]:
Math(vlatex(eq.fx_eq))

<IPython.core.display.Math object>

In [10]:
Math(vlatex(eq.fy_eq))

<IPython.core.display.Math object>

In [11]:
Math(vlatex(eq.mz_eq))

<IPython.core.display.Math object>

In [12]:
Math(vlatex(eq.X_eq))

<IPython.core.display.Math object>

In [13]:
Math(vlatex(eq.Y_eq))

<IPython.core.display.Math object>

In [14]:
Math(vlatex(eq.N_eq))

<IPython.core.display.Math object>

In [15]:
r1d_eq = sp.Eq(r.diff(), sp.solve(N_eq, r.diff())[0])

## Steady state solution

In [16]:
X_steady = X_eq.subs([
    (u.diff(),0),
    (v.diff(),0),
    (r.diff(),0),
          ])
X_steady

Eq(0, X_{delta}*delta(t) + X_{r}*r(t) + X_{u}*u(t) + X_{v}*v(t))

In [17]:
Y_steady = Y_eq.subs([
    (u.diff(),0),
    (v.diff(),0),
    (r.diff(),0),
          ])
Y_steady

Eq(U*m*r(t), Y_{delta}*delta(t) + Y_{r}*r(t) + Y_{u}*u(t) + Y_{v}*v(t))

In [18]:
N_steady = N_eq.subs([
    (u.diff(),0),
    (v.diff(),0),
    (r.diff(),0),
          ])
N_steady

Eq(U*m*x_G*r(t), N_{delta}*delta(t) + N_{r}*r(t) + N_{u}*u(t) + N_{v}*v(t))

In [19]:
eqs = [Y_steady,N_steady]
solution = sp.solve(eqs,v,r, dict=True)
solution[0][v]

-N_{delta}*U*m*delta(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) + N_{delta}*Y_{r}*delta(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) - N_{r}*Y_{delta}*delta(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) - N_{r}*Y_{u}*u(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) - N_{u}*U*m*u(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) + N_{u}*Y_{r}*u(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) + U*Y_{delta}*m*x_G*delta(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) + U*Y_{u}*m*x_G*u(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G)

In [20]:
solution[0][r]

-N_{delta}*Y_{v}*delta(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) - N_{u}*Y_{v}*u(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) + N_{v}*Y_{delta}*delta(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G) + N_{v}*Y_{u}*u(t)/(N_{r}*Y_{v} + N_{v}*U*m - N_{v}*Y_{r} - U*Y_{v}*m*x_G)

In [21]:
subs = {value:key for key,value in eq.p.items()}
#u_lambda = lambdify(solution[0][u].subs(subs))
v_lambda = lambdify(solution[0][v].subs(subs))
r_lambda = lambdify(solution[0][r].subs(subs))

## Prime system

In [22]:
prime_system.df_prime.loc[['denominator']]

,length,volume,mass,density,inertia_moment,time,area,angle,-,linear_velocity,angular_velocity,linear_acceleration,angular_acceleration,force,moment
denominator,L,L**3,0.5*L**3*rho,0.5*rho,0.5*L**5*rho,L/U,L**2,1,1,U,U/L,U**2/L,U**2/L**2,0.5*L**2*U**2*rho,0.5*L**3*U**2*rho


## Ship parameters

In [23]:
T_ =10
L_ = 200
CB_ = 0.7
B_ = 30
rho_ = 1025
m_ = T_*B_*L_*CB_*rho_

ship_parameters = {
        'T' : T_,
        'L' : L_,
        'CB' :CB_,
        'B' : B_,
        'rho' : rho_,
        'x_G' : 0,
        'm' : m_,
        'I_z': 0.2*m_*L_**2, 
    }

ps = prime_system.PrimeSystem(**ship_parameters)


In [24]:
def calculate_prime(row, ship_parameters):
    return run(function=row['brix_lambda'], inputs=ship_parameters)

mask = df_parameters['brix_lambda'].notnull()
df_parameters.loc[mask,'prime'] = df_parameters.loc[mask].apply(calculate_prime, ship_parameters=ship_parameters, axis=1)
df_parameters.loc['Ydelta','prime'] = 0.0005  # Just guessing
df_parameters.loc['Ndelta','prime'] = -df_parameters.loc['Ydelta','prime']/4  # Just guessing

ValueError: Sympy lambda function misses:['volume']

In [ ]:
df_parameters['prime'].fillna(0, inplace=True)
df_parameters

## Steady turning

In [ ]:
inputs = dict(df_parameters['prime'])
inputs.update(ps.prime(ship_parameters))

U = 10
delta = np.deg2rad(35)
inputs['delta'] = delta
inputs['u'] = inputs['U'] =  ps._prime(U,'linear_velocity', U=U)


v_steady_prime = run(function=v_lambda, inputs=inputs)
r_steady_prime = run(function=r_lambda, inputs=inputs)

v_steady = ps._unprime(v_steady_prime,'linear_velocity', U=U)
r_steady = ps._unprime(r_steady_prime,'angular_velocity' ,U=U)


In [ ]:
v_steady

In [ ]:
r_steady

In [ ]:
model.u1d_lambda

In [ ]:
model.v1d_lambda

In [ ]:
states = {
    'u' : ps._prime(U,'linear_velocity',U=U),
    'v' : ps._prime(v_steady,'linear_velocity',U=U),
    'r' : ps._prime(r_steady,'angular_velocity',U=U),
    #'u1d' : 0,
    #'v1d' : 0,
    #'r1d' : 0,
    
}

control = {
    'delta' : delta,
    'U' : ps._prime(U,'linear_velocity',U=U)
}

inputs = dict(df_parameters['prime'])
inputs.update(ps.prime(ship_parameters))

inputs.update(states)
inputs.update(control)

run(function=model.v1d_lambda, inputs=inputs)

In [ ]:
run(function=model.u1d_lambda, inputs=inputs)

In [ ]:
run(function=model.v1d_lambda, inputs=inputs)

In [ ]:
run(function=model.r1d_lambda, inputs=inputs)

In [ ]:
t_max = 2*np.pi/r_steady  # Turn one lap
t = np.arange(0,t_max,0.01)


control = {
    'delta' : delta,
}

parameters = dict(df_parameters['prime'])

u_steady = np.sqrt(U**2 - v_steady**2)
y0 = {
    'u' : u_steady, 
    'v' : v_steady,
    'r' : r_steady,
    'x0' : 0,
    'y0' : 0,
    'psi' : 0,
    }

#solution = model.simulate(y0=y0, t=t, df_parameters=df_parameters, df_ship_parameters=df_ship_parameters, control=control, rtol=1e-3, atol=1e-3)
solution = model.simulate(y0=y0, t=t, df_parameters=df_parameters, ship_parameters=ship_parameters, control=control)

columns = ['u','v','r','x0','y0','psi']
df_result_prime = pd.DataFrame(data=solution.y.T, columns=columns)
df_result_prime.index=t[0:len(df_result_prime)]

#df_result_prime['U'] = np.sqrt(df_result_prime['u']**2 + df_result_prime['v']**2)

df_result_prime.plot(y='u')
df_result_prime.plot(y='v')
df_result_prime.plot(y='r')



In [ ]:
y0

In [ ]:
solution

In [ ]:
#units = {key:value[1] for key,value in zip(columns,y0)}
df_result = ps.unprime(values=df_result_prime, U=U)


df_result['beta'] = -np.arctan2(df_result['v'],df_result['u'])

In [ ]:
df_result.head()

In [ ]:
track_plot(df=df_result, lpp=ship_parameters['L'], beam=ship_parameters['B'])

df_result.plot(y='u')
df_result.plot(y='v')
df_result.plot(y='r')
df_result.plot(y='psi')
df_result.plot(y='beta')



## Steady turnings

In [ ]:
inputs = dict(df_parameters['prime'])
inputs.update(ps.prime(ship_parameters))

U = 10
delta = np.deg2rad(35)
inputs['delta'] = delta
inputs['u'] = inputs['U'] =  ps._prime(U,'linear_velocity', U=U)

s = pd.Series(inputs)
N=10
data = np.tile(s.values,(N,1))
df_variation = pd.DataFrame(data, columns=s.index)
df_variation.head()
df_variation['delta'] = np.linspace(0.01,0.3,N)

df_variation['v'] = run(function=v_lambda, inputs=df_variation)
df_variation['r'] = run(function=r_lambda, inputs=df_variation)


In [ ]:
df_variation.plot(x='delta', y='r')
df_variation.plot(x='delta', y='v')

